# 連想配列と連結リスト


## キーポイント

*


----

## 1 unordered_set(アンオーダード・セット)

----


### 1.1 find関数の比較回数

「文章に特定の単語が含まれているか調べるプログラム」を作ることを考えます。<br>
思いつく方法として、次のようなものがあります。

1. `vector<string>`に単語単位で文章を読み込む
2. `find`関数で単語を探す

この方針で作成したプログラムを次に示します。

In [ ]:
# @title コード
%%writefile find_word_using_find.cpp
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <algorithm>
using namespace std;

int main() {
  vector<string> v;
  v.reserve(1000);

  fstream ifs("sample.txt");
  for (;;) {
    string s;
    ifs >> s;
    if (ifs.eof()) {
      break;
    }
    v.push_back(s);
  }

  string x;
  cin >> x;

  auto i = ranges::find(v, x);

  if (i != v.end()) {
    cout << x << " is found." << endl;
  } else {
    cout << x << " is not found." << endl;
  }
}

In [ ]:
# @title 実行テスト
!echo "The names John Doe for males, Jane Doe or Jane Roe for females, or Jonnie Doe and Janie Doe for children, or just Doe non-gender-specifically are used as placeholder names for a party whose true identity is unknown or must be withheld in a legal action, case, or discussion. The names are also used to refer to acorpse or hospital patient whose identity is unknown. This practice is widely used in the United States and Canada, but is rarely used in other English-speaking countries including the United Kingdom itself, from where the use of John Doe in a legal context originates. The names Joe Bloggs or John Smith are used in the UK instead, as well as in Australia and New Zealand.">sample.txt
!echo "John Doe is sometimes used to refer to a typical male in other contexts as well, in a similar manner to John Q. Public, known in Great Britain as Joe Public, John Smith or Joe Bloggs. For example, the first name listed on a form is often John Doe, along with a fictional address or other fictional information to provide an example of how to fill in the form. The name is also used frequently in popular culture, for example in the Frank Capra film Meet John Doe. John Doe was also the name of a 2002 American television series.">>sample.txt
!echo "Similarly, a child or baby whose identity is unknown may be referred to as Baby Doe. A notorious murder case in Kansas City, Missouri, referred to the baby victim as Precious Doe. Other unidentified female murder victims are Cali Doe and Princess Doe. Additional persons may be called James Doe, Judy Doe, etc. However, to avoid possible confusion, if two anonymous or unknown parties are cited in a specific case or action, the surnames Doe and Roe may be used simultaneously; for example, John Doe v. Jane Roe. If several anonymous parties are referenced, they may simply be labelled John Doe #1, John Doe #2, etc. (the U.S. Operation Delego cited 21 (numbered) John Doe s) or labelled with other variants of Doe / Roe / Poe / etc. Other early alternatives such as John Stiles and Richard Miles are now rarely used, and Mary Major has been used in some American federal cases." >> sample.txt
!g++ -std=c++20 -O2 -Wall -Wextra -o find_word_using_find find_word_using_find.cpp && echo "この下をクリックして検索したい単語を入力" && ./find_word_using_find

`find`関数による検索は、基本的にはfor文と同じで、範囲の先頭からひとつずつ順番に、条件に合うかどうかを調べます。<br>
つまり、`find`関数を使った次のプログラムは、

```cpp
auto i = ranges::find(v, x);
```

for文を使った次のプログラムと同じです。

```cpp
auto i = v.begin();
for (; i != v.end(); i++) {
  if (*i == x) {
    break;
  }
}
```

`find`関数とfor文に共通の問題は「データ量に比例して検索に時間がかかるようになる」ことです。<br>
データがランダムな位置で見つかると仮定すると、データを見つけるまでに平均して`データ数 / 2`回の比較が行われます。<br>
例えば、単語の数が10万語の文章が入力された場合、平均比較回数は5万回です。

2025年現在のコンピューターであれば、5万回程度の比較は1秒とかからず完了するでしょう。ですが、ゲームのように1/60秒以内に様々な処理を行わなくてはならないアプリや、多くの人から閲覧されて大量のリクエストを受けるWebサイトでは、もっと短い時間で処理を終了させなくてはなりません。

そのためには、`find`やfor文より高速な検索方法を使う必要があります。


### 1.2 連想配列の概要

検索を高速化する方法として、適切なコンテナを使うことが挙げられます。<br>
例えば、「連想配列(れんそうはいれつ)」という種類のコンテナを使うと、かなり少ない比較回数でデータを見つけられます。

連想配列は、簡単に言うと「どんなデータでも添え字にできる配列」です。連想配列の添字は「キー(鍵)」と呼ばれます。<br>
連想配列という名前は、「キーからデータを即座に連想(取得)できる」というイメージから来ています。

連想配列の特徴は以下の2点です。

* どんなデータでも添字(キー)にできる
* データの格納方法が工夫されていて、データの検索・追加・削除がとても速い

>**【キー(鍵)】**<br>
>連想配列では、添え字のことを「キー(鍵)」と呼びます(添え字が「数字」とは限らないため)。<br>
>本テキストでは、キーと書かれていたら「連想配列の添え字」を意味するものとします。

これらの特徴が役立つ場合は連想配列を使い、特にこれらの特徴が不要な場合は`vector`を使うとよいでしょう。

2025年現在、C++の連想配列は全部で8種類あります。

| 名前 | 性質 | 格納する要素 | キーが等しい要素 | 要素の格納方法 | 追加時期 |
|:-----|:-----|:---------------|:------------------------:|:-----------------|:---------|
| <font size=3>unordered_map</font><br>(アンオーダード・マップ) | <font size=2>検索・追加・削除が非常に高速。<br>データの格納順は不定。</font> | キーとデータ | 禁止 | ハッシュ | C++11 |
| <font size=3>unordered_set</font><br>(アンオーダード・セット) | &emsp;(同上) | キー | 禁止 | ハッシュ | C++11 |
| <font size=3>unordered_multimap</font><br>(アンオーダード・マルチ・マップ) | &emsp;(同上) | キーとデータ| 許可 | ハッシュ | C++11 |
| <font size=3>unordered_multiset</font><br>(アンオーダード・マルチ・セット) | &emsp;(同上) | キー | 許可 | ハッシュ | C++11 |
| <font size=3>map</font>(マップ) | <font size=2>検索・追加・削除が高速。<br>データの格納順は一定。</font> | キーとデータ | 禁止 | 平衡二分木 | 初期から存在 |
| <font size=3>set</font>(セット) | &emsp;(同上) | キー | 禁止 | 平衡二分木 | 初期から存在 |
| <font size=3>multimap</font>(マルチ・マップ) | &emsp;(同上) | キーとデータ | 許可 | 平衡二分木 | 初期から存在 |
| <font size=3>multiset</font>(マルチ・セット) | &emsp;(同上) | キー | 許可 | 平衡二分木 | 初期から存在 |

これらの連想配列型は、「初期からある型」と、C++11で追加された「名前に`unordered`(アンオーダード)が付く型」に大別できます。<br>
一般的には、後から追加された「名前に`unordered`が付く型」のほうが性能が良いです。<br>
さらに、「名前に`multi`が付く型」は利用頻度が低めです。

そこで、本テキストでは性能も利用頻度も高めの`unordered_set`型と`unordered_map`型の2つに絞って解説します。


### 1.3 unordered_setの概要

`unordered_set`型は、「キーだけを持つ連想配列」です。「データをキーにしたい」場合に使用します。<br>
`unorderd_set`というヘッダファイルをインクルードすることで使用できます。

`unordered_set`型は、テンプレートを使って次のように宣言されています。

```cpp
template<typename Key>
class unordered_set;
```

>`class`(クラス)キーワードを使うと、`struct`キーワードと同じように型を宣言できます。詳細は回を改めて説明します。

このように、「テンプレートを使って宣言された型」のことを「クラス・テンプレート」といいます。

さて、少し上で書いた`vector`型を使ったプログラムを、`unordered_set`型を使って置き換えると、次のようになります。


In [ ]:
# @title コード
%%writefile find_word_using_unordered_set.cpp
#include <iostream>
#include <fstream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v;
  v.reserve(10'000);

 fstream ifs("sample.txt");
  for (;;) {
    string s;
    ifs >> s;
    if (ifs.eof()) {
      break;
    }
    v.insert(s);
  }

  string x;
  cin >> x;

  auto i = v.find(x);

  if (i != v.end()) {
    cout << "Yes" << endl;
  } else {
    cout << "No" << endl;
  }
}


In [ ]:
# @title 実行テスト
!echo "The names John Doe for males, Jane Doe or Jane Roe for females, or Jonnie Doe and Janie Doe for children, or just Doe non-gender-specifically are used as placeholder names for a party whose true identity is unknown or must be withheld in a legal action, case, or discussion. The names are also used to refer to acorpse or hospital patient whose identity is unknown. This practice is widely used in the United States and Canada, but is rarely used in other English-speaking countries including the United Kingdom itself, from where the use of John Doe in a legal context originates. The names Joe Bloggs or John Smith are used in the UK instead, as well as in Australia and New Zealand.">sample.txt
!echo "John Doe is sometimes used to refer to a typical male in other contexts as well, in a similar manner to John Q. Public, known in Great Britain as Joe Public, John Smith or Joe Bloggs. For example, the first name listed on a form is often John Doe, along with a fictional address or other fictional information to provide an example of how to fill in the form. The name is also used frequently in popular culture, for example in the Frank Capra film Meet John Doe. John Doe was also the name of a 2002 American television series.">>sample.txt
!echo "Similarly, a child or baby whose identity is unknown may be referred to as Baby Doe. A notorious murder case in Kansas City, Missouri, referred to the baby victim as Precious Doe. Other unidentified female murder victims are Cali Doe and Princess Doe. Additional persons may be called James Doe, Judy Doe, etc. However, to avoid possible confusion, if two anonymous or unknown parties are cited in a specific case or action, the surnames Doe and Roe may be used simultaneously; for example, John Doe v. Jane Roe. If several anonymous parties are referenced, they may simply be labelled John Doe #1, John Doe #2, etc. (the U.S. Operation Delego cited 21 (numbered) John Doe s) or labelled with other variants of Doe / Roe / Poe / etc. Other early alternatives such as John Stiles and Richard Miles are now rarely used, and Mary Major has been used in some American federal cases." >> sample.txt
!g++ -std=c++20 -O2 -Wall -Wextra -o find_word_using_unordered_set find_word_using_unordered_set.cpp && echo "この下をクリックして検索したい単語を入力" && ./find_word_using_unordered_set

`unordered_set`にデータを追加するには、`insert`(インサート)メンバ関数を使います。<br>
データの検索には、`unordered_set`自身の`find`メンバ関数を使います(汎用アルゴリズムの`find`関数は使いません)。

`unordered_set`の`find`メンバ関数は`unordered_set`専用の特別な設計になっていて、どれほどデータ数が多くなっても、ほぼ1回の比較で目的のデータを見つけられます。


### 1.4 キーの有無を調べる

`unordered_set`に「特定のキー」が存在することを調べるには、`find`(ファインド)メンバ関数を使います。

>**書式**
>
>```cpp
>iterator find(const Key& key);
>```
>
>**引数**
>
>* key&emsp;検索するキー
>
>**戻り値**
>
>引数`key`と一致するキーが見つかれば、そのキーを差すイテレータを返します。<br>
>見つからなければ、`end`メンバ関数の戻り値を返します。

`find`関数の戻り値は双方向イテレータです。イテレータを「参照外し」することで、データを読み書きできます。

もしキーが見つからなかった場合、戻り値のイテレータは、`unordered_set`の`end`メンバ関数の値と一致します。

次のプログラムは、`find`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v = { "andy", "barbie", "conrad", "davis", "elli", "finn" };

  auto i = v.find("gaia");
  if (i == v.end()) {
    cout << "gaiaはありません" << endl;
  } else {
    cout << *i << "はあります" << endl;
  }

  i = v.find("conrad");
  if (i == v.end()) {
    cout << "conradはありません" << endl;
  } else {
    cout << *i << "はあります" << endl;
  }
}
```

**実行結果**

```txt
gaiaはありません
conradはあります
```



### 1.5 キーを追加する

`unordered_set`にキーを追加するには、`emplace`(エンプレイス)メンバ関数を使います。

>**書式**
>
>```cpp
>pair<iterator, bool> emplace(const Key& key);
>```
>
>**引数**
>
>* key&emsp;追加するキー
>
>**戻り値**
>
>引数`key`と一致するキーがなければ、追加したキーを差すイテレータと`true`のペアを返します。<br>
>一致するキーがあれば、そのキーを指すイテレータと`false`のペアを返します。

このメンバ関数の戻り値は「追加したデータの位置をあらわすイテレータ」と、「追加の結果をあらわす真偽値」のペアです。


#### pair(ペア)構造体

`emplace`メンバ関数は2つの値を返す必要があるため、標準ライブラリにある`pair`(ペア)型を使っています。<br>
`pair`型は次のような、「テンプレート引数によって、2つのメンバ変数の型を指定できる構造体」として定義されています。

```cpp
template<typename T, typename U>
struct pair {
  T  first;  // １個目のデータ
  U  second; // ２個目のデータ
};
```

例えば、`pair<iterator, bool>`型は次のような構造体として扱われます。

```cpp
struct pair<iterator, bool> {
  iterator first;
  bool     second;
};
```


`emplace`関数の戻り値は、キーの追加に成功したか失敗したかで、次のように変化します。

| 追加の成否 | firstの内容 | secondの内容 |
|:----:|:------|:-------|
| 成功 | 追加したキーを指すイテレータ | true |
| 失敗 | 追加しようとしたキーと同じキーを指すイテレータ | false |

追加の成否は、`second`メンバ変数を調べることで確認できます。

次のプログラムは、`empalce`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v = { "andy", "barbie", "conrad", "davis", "elli", "finn" };

  // キーがgaiaの要素はないので成功する(secondにtrueが代入される)
  auto a = v.emplace("gaia");
  if (a.second) {
    cout << *a.first << "を追加" << endl;
  } else {
    cout << "gaiaの追加に失敗" << endl;
  }

  // キーがdavisの要素はすでにあるので失敗する(secondにfalseが代入される)
  auto b = v.emplace("davis");
  if (b.second) {
    cout << *a.first << "を追加" << endl;
  } else {
    cout << "davisの追加に失敗" << endl;
  }
}
```

**実行結果**

```txt
gaiaを追加
davisの追加に失敗
```


### 1.6 キーを削除する

`unordered_set`からデータを削除するには、`erase`(イレース)メンバ関数を使います。<br>
設計上の理由から、汎用アルゴリズムの`erase`は連想配列に対応していないため、間違って使用するとコンパイルエラーになります。<br>
注意してください(ただし、`erase_if`汎用アルゴリズムは使えます)。

>**書式**
>
>```cpp
>size_type erase(const Key& key);
>```
>
>**引数**
>
>* key&emsp;削除するキー
>
>**戻り値**
>
>引数`key`と一致するキーがなければ`0`を返します。<br>
>一致するキーがあれば`1`を返します。

`erase`メンバ関数の戻り値は「削除したキーの数」です。この値は、削除に成功したら`1`、失敗したら`0`になります。

>**【size_typeについて】**<br>
>`size_type`(サイズ・タイプ)型は、コンテナ(ここでは`unordered_set`)ごとに宣言されている「大きさをあらわす型」です。ほぼすべての環境で`size_t`型の別名なので、`size_t`型だと考えて差し支えありません。

次のプログラムは、`erase`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_set>
#include <string>
using namespace std;

int main() {
  unordered_set<string> v = { "andy", "barbie", "conrad", "davis", "elli", "finn" };

  if (v.erase("andy")) {
    cout << "andyを削除" << endl;
  } else {
    cout << "andyを削除できません" << endl;
  }

  auto i = v.find("andy");
  if (i == v.end()) {
    cout << "andyはありません" << endl;
  } else {
    cout << "andyはあります" << endl;
  }
}
```

**実行結果**

```txt
andyを削除
andyはありません
```


----

## 2 unordered_map(アンオーダード・マップ)

----


### 2.1 unordered_mapの概要

`unordered_map`型は、「キーとデータをペアにして持つ連想配列」です。「キーとデータが異なる」場合に使用します。<br>
`unordered_map`というヘッダファイルをインクルードすることで使用できます。

`unordered_map`型は、テンプレートを使って次のように宣言されています。

```cpp
template<typename Key, typename Value>
class unordered_map;
```

`unordered_map`が使われるのは、例えば「名前に対応するデータを検索したい」場合です。<br>
次のプログラムは、`unordered_map`を使って、「名前」と「得点」を関連付ける例です。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  // []演算子を使ってデータを読み書きできる
  string s = "barbie";
  cout << s << "の得点は" << v[s] << "点です" << endl;
}
```

**実行結果**

```txt
barbieの得点は75点です
```

上記のプログラムで、初期化の`{}`の中身が、キーとデータのペアになっている点に注目してください。<br>
例えば、最初のペアは`{"andy", 60}`で、キーは`"andy"`、データは`60`となります。

また、`unordered_map`は`[]`演算子にキーを指定することで、キーに対応するデータを読み書きできます。


### 2.2 unorderd_mapとpair構造体

連想配列は、キーとデータをペアにして格納します。これは、標準ライブラリの`pair`(ペア)構造体によって実現しています。

`pair`構造体は次のように、テンプレートとして定義されています。

```cpp
template<typename T, typename U>
struct pair {
  T  first;  // １個目のデータ
  U  second; // ２個目のデータ
};
```

例えば、`unordered_map<char, string>`型が使用するペアは、次のようになります。

```cpp
struct pair<char, string> {
  char   first;
  string second;
};
```

キーは`first`(ファースト)メンバ変数に格納されます。<br>
データは`second`(セカンド)メンバ変数に格納されます。


### 2.3 キーの有無を調べる

連想配列は何でもキーにできるため、「キーに対応するデータがない」場合のことも考えなくてはなりません。

例えば、上記のプログラムでキーを`barbie`から`gaia`に変えたとします。しかし、連想配列の中にキーが`gaia`のデータはありません。<br>
そのため、正しいデータを取得できません。

あるキーが存在することを調べるには、`find`(ファインド)メンバ関数を使います。

>**書式**
>
>```cpp
>iterator find(const Key& key);
>```
>
>**引数**
>
>* key&emsp;検索するキー
>
>**戻り値**
>
>引数`key`と一致するキーが見つかれば、そのキーを差すイテレータを返します。<br>
>見つからなければ、`end`メンバ関数の戻り値を返します。

`find`関数の戻り値は双方向イテレータです。<br>
`unrodered_set`と異なり、`unordered_map`のイテレータは「キーとデータのペア」の位置をあらわします。<br>
そのため、`->`演算子を使ってペアのメンバ変数を読み書きします。

もしキーが見つからなかった場合、戻り値のイテレータは`unordered_map`の`end`メンバ関数の値と一致します。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  string s = "gaia";
  auto i = v.find(s);

  if (i == v.end()) {
    cout << s << "という名前は登録されていません" << endl;
  } else {
    cout << i->first << "の得点は" << i->second << "点です" << endl;
  }

  s = "elli";
  i = v.find(s);

  if (i == v.end()) {
    cout << s << "という名前は登録されていません" << endl;
  } else {
    cout << i->first << "の得点は" << i->second << "点です" << endl;
  }
}
```

**実行結果**

```txt
gaiaという名前は登録されていません
elliの得点は90点です
```


### 2.4 データの読み書き

`[]`(添え字)演算子の添え字としてキーを指定すると、キーに関連付けられたデータを読み書きできます。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  cout << "andyの現在の得点 : " << v["andy"] << endl;

  v["andy"] = 70;

  cout << "andyの新しい得点 : " << v["andy"] << endl;
}
```

**実行結果**

```txt
andyの現在の得点 : 60
andyの新しい得点 : 70
```


### 2.5 キーとデータを追加する

`unordered_map`にデータを追加するには`try_emplace`(トライ・エンプレイス)メンバ関数を使います。

>**書式**
>
>```cpp
>pair<iterator, bool> try_emplace(const Key& ket, const Vaule& value);
>```
>
>**引数**
>
>* key&emsp;&nbsp;&nbsp;&nbsp;追加するキー
>* value&emsp;追加するデータ
>
>**戻り値**
>
>引数`key`と一致するキーがなければ、追加したキーを差すイテレータと`true`のペアを返します。<br>
>一致するキーがあれば、そのキーを指すイテレータと`false`のペアを返します。

`try_emplace`メンバ関数の戻り値は、「追加したキーの位置をあらわすイテレータ」と、「追加の結果をあらわす真偽値」のペアです。<br>
ペアの値は、キーの追加に成功したか失敗したかで、次のように変化します。

| 追加の成否 | firstの内容 | secondの内容 |
|:----:|:------|:-------|
| 成功 | 追加したキーを指すイテレータ | true |
| 失敗 | 追加しようとしたキーと同じキーを指すイテレータ | false |

追加の成否は、`second`メンバ変数を調べることで確認できます。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  // キーがgaiaの要素はないので成功する
  auto a = v.try_emplace("gaia", 40);
  if (a.second) {
    auto i = a.first; // イテレータを取得
    cout << i->second << "の得点を追加" << endl;
  } else {
    cout << "gaiaの追加に失敗" << endl;
  }

  // キーがgaiaの要素はすでにあるので失敗する
  auto b = v.try_emplace("gaia", 50);
  if (b.second) {
    cout << b.first->second << "を追加" << endl;
  } else {
    cout << "gaiaの追加に失敗" << endl;
  }

  // キーがgaiaの要素を検索
  auto i = v.find("gaia");
  if (i == v.end()) {
    cout << i->first << "の得点: " << i->second << endl;
  } else {
    cout << "gaiaという名前は登録されていません" << endl;
  }
}
```

**実行結果**

```txt
gaiaの得点を追加
gaiaの追加に失敗
gaiaの得点: 40
```


### 2.6 []演算子でキーとデータを追加する

あまり推奨されませんが、`[]`演算子を使うことでも、キーとデータを追加できます。

`[]`演算子には、

&emsp;**キーが存在しない場合は自動的にキーを追加し、「空のデータ」を設定する**

という機能があるからです。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v;

  auto i = v.find("andy");
  if (i != v.end()) {
    cout << "andyは登録済みです(得点:" << i->second << ")" << endl;
  } else {
    cout << "andyは未登録です" << endl;
  }

  v["andy"] = 60;
  cout << "andyを登録" << endl;

  i = v.find("andy");
  if (i != v.end()) {
    cout << "andyは登録済みです(得点:" << i->second << ")" << endl;
  } else {
    cout << "andyは未登録です" << endl;
  }
}
```

**実行結果**

```txt
andyは未登録です
andyを登録
andyは登録済みです(得点:60)
```

つまり、「`[]`演算子は追加と更新の両方の機能を持つ」わけです。<br>
便利な機能ですが、「更新したいときに間違ったキーを指定してしまうと、新しいキーが作られてしまう」という問題もあります。

そのため、データを更新するとき、キーが確実に存在すると分かっているなら`[]`演算子を使い、不明な場合は`find`メンバ関数を使って、キーの有無を確認するとよいでしょう。

また、データを追加するときは、`[]`演算子より`try_emplace`メンバ関数を優先してください。<br>
`[]`演算子では、「追加したい」のか「更新したい」のかが分かりにくいためです。<br>
その点、`try_empalce`は、見ただけで「追加したい」ということが分かります。

### 2.7 キーとデータを削除する

`unordered_map`からキーとデータを削除するには`erase`(イレース)メンバ関数を使います。<br>
なお、設計上の理由から、汎用アルゴリズムの`erase`は連想配列に対応していないので使えないので、注意してください<br>
(ただし、`erase_if`汎用アルゴリズムは使えます)。

>**書式**
>
>```cpp
>size_type erase(const Key& key);
>```
>
>**引数**
>
>* key&emsp;削除するキー
>
>**戻り値**
>
>引数`key`と一致するキーがなければ`0`を返します。<br>
>一致するキーがあれば`1`を返します。

`erase`メンバ関数の戻り値は「削除したペアの数」です。この値は、削除に成功したら`1`、失敗したら`0`になります。

次のプログラムは、`erase`メンバ関数の使用例です。

**コード**

```cpp
#include <iostream>
#include <unordered_map>
#include <string>
using namespace std;

int main() {
  unordered_map<string, int> v = { {"andy", 60}, {"barbie", 75}, {"conrad", 57}, {"davis", 90}, {"elli", 90}, {"finn", 80} };

  auto i = v.find("conrad");
  if (i == v.end()) {
    cout << "conradは登録されていません" << endl;
  } else {
    cout << i->first << "の得点: " << i->second << endl;
  }

  if (v.erase("conrad")) {
    cout << "conradを削除" << endl;
  } else {
    cout << "conradを削除できません" << endl;
  }

  i = v.find("conrad");
  if (i == v.end()) {
    cout << "conradは登録されていません" << endl;
  } else {
    cout << i->first << "の得点: " << i->second << endl;
  }
}
```

**実行結果**

```txt
conradの得点: 57
conradを削除
conradは登録されていません
```



### 2.8 連想配列を「空きの多い配列」として使う

>ここは少し難しい話です。今はまだ分からなくても、あまり気にしないでください。

$100m^2$ の平面空間に、100体の敵がいて、敵同士が重ならないようにしたいとします。

敵同士の衝突判定は、1体につき残り99体の敵との衝突判定を行うことで実現できます。この方法では、9900回の衝突判定を行う必要があります。

しかし、「衝突するほど近い距離にいる敵の数」は、多くても10体くらいでしょう。<br>
ということは、「衝突するほど近い敵」を判別できれば、衝突判定の回数を、1000回程度まで減らせるはずです。

このような場合、例えば $100m^2$ 空間を $5m^2$ 単位に分割して、20x20マスの配列を作ります。すべての敵を「位置に対応するマス」に登録します。<br>
このデータ構造では、同じマスにいる敵は「衝突するほど近い敵」と判定できます。

**配列の例(4x4マスに8体の敵がいる)**

```txt
   0   5  10  15  20
 0┌─┬─┬─┬─┐    v[0]  = { 敵1, 敵2 }
  │ 0│ 1│ 2│ 3│    v[1]  = {}
 5├─┼─┼─┼─┤    v[2]  = { 敵6 }
  │ 4│ 5│ 6│ 7│    v[3]  = {}
10├─┼─┼─┼─┤    v[4]  = { 敵4, 敵8 }
  │ 8│ 9│10│11│    v[5]  = {}
15├─┼─┼─┼─┤    v[6]  = {}
  │12│13│14│15│    v[7]  = {}
20└─┴─┴─┴─┘    v[8]  = {}
                        v[9]  = {}
                        v[10] = { 敵3 }
                        v[11] = {}
                        v[12] = {}
                        v[13] = {}
                        v[14] = {}
                        v[15] = { 敵5, 敵7 }
```

**コード**

```cpp
#include <iostream>
#include <vector>
using namespace std;

struct Enemy {
  bool HitCheck(const Enemy& other) {
    return abs(x - other.x) <= 1 && abs(y - other.y) <= 1;
  }

  int ToIndex() { return (y / 5) * 4 + (x / 5); }

  int x, y;
};

int main() {
  // 敵の配列
  vector<Enemy> enemies = { { 2, 3}, { 4, 1}, {13,13}, { 3, 7},
                            {16,17}, {11, 2}, {18,16}, { 3, 8} };
  // 敵をマス目に登録
  vector<vector<Enemy*>> v(16);
  for (int i = 0; i < (int)enemies.size(); i++) {
    int index = enemies[i].ToIndex();
    v[index] = &enemies[i];
  }

  // マス目の敵の衝突判定
  for (int i = 0; i < (int)v.size(); i++) {
    for (int a = 0; a < (int)v[i].size() - 1; a++) {
      Enemy* pa = v[i][a];
      for (int b = a + 1; b < (int)v[i].size(); b++) {
        Enemy* pb = v[i][b];
        if (pa->HitCheck(*pb)) {
          cout << pa->x << ',' << pa->y << ' ' << pb->x << ',' << pb->y << endl;
        }
      } // for b
    } // for a
  } // for i
}
```

**出力結果**

```txt
3,7 3,8
```

>この例では省略していますが、本来、面積を持つ物体の衝突判定では、周囲のマスの敵とも衝突判定を行う必要があります。

２次元配列を使う場合の問題点は、「敵のいないマスのメモリが無駄になる」ということです。<br>
例えば、上の図のように16マス中5マスしか敵がいない場合、11マスは全く使われません。

このような、「空きが多い配列」を効率的に表現するには、連想配列を使います。<br>
例えば、敵のいるマスの番号をキーとする連想配列を作ります。そして、敵のいるマスだけを連想配列に追加します。

**連想配列の例(4x4マスに8体の敵がいる)**

```txt
   0   5  10  15  20
 0┌─┬─┬─┬─┐    m[0]  = { 敵1, 敵2 }
  │ 0│ 1│ 2│ 3│    m[2]  = { 敵6 }
 5├─┼─┼─┼─┤    m[4]  = { 敵4, 敵8 }
  │ 4│ 5│ 6│ 7│    m[10] = { 敵3 }
10├─┼─┼─┼─┤    m[15] = { 敵5, 敵7 }
  │ 8│ 9│10│11│
15├─┼─┼─┼─┤
  │12│13│14│15│
20└─┴─┴─┴─┘
```

**コード**

```cpp
#include <iostream>
#include <vector>
#include <unordered_map>
using namespace std;

struct Enemy {
  bool HitCheck(const Enemy& other) {
    return abs(x - other.x) <= 1 && abs(y - other.y) <= 1;
  }

  int ToIndex() { return (y / 5) * 4 + (x / 5); }

  int x, y;
};

int main() {
  // 敵の配列
  vector<Enemy> enemies = { { 2, 3}, { 4, 1}, {13,13}, { 3, 7},
                            {16,17}, {11, 2}, {18,16}, { 3, 8} };
  // 敵をマス目に登録
  unordered_map<int, vector<Enemy*>> m;
  for (int i = 0; i < (int)enemies.size(); i++) {
    int index = enemies[i].ToIndex();
    m[index].push_back(&enemies[i]);
  }

  // マス目の敵の衝突判定
  for (auto i = m.begin(); i != m.end(); i++) {
    for (int a = 0; a < (int)i->second.size() - 1; a++) {
      Enemy* pa = i->second[a];
      for (int b = a + 1; b < (int)i->second.size(); b++) {
        Enemy* pb = i->second[b];
        if (pa->HitCheck(*pb)) {
          cout << pa->x << ',' << pa->y << ' ' << pb->x << ',' << pb->y << endl;
        }
      } // for b
    } // for a
  } // for i
}
```

**出力結果**

```txt
3,7 3,8
```

連想配列に追加されるマスは0, 2, 4, 10, 15の5つだけなので、敵のいないマスのために無駄なメモリを使わなくて済みます。

>**【ひとつのキーに複数のデータを割り当てる他の方法】**<br>
>連想配列の例では、複数のデータを割り当てるために配列を使いました。<br>
>他の方法として、`unordered_multimap`(アンオーダード・マルチマップ)というクラスを使うこともできます。<br>
>名前に「マルチ」とあるように、アンオーダード・マルチマップは、同じキーに対して複数のデータを割り当てられます。

>**【キーとデータが同じ場合】**<br>
>キーとデータが等しい、つまりキーだけで十分な場合、`unordered_set`(アンオーダード・セット)クラスを使います。<br>
>アンオーダード・セットはキーしか持たない連想配列です。



----

## 3 list(リスト)

----


### 2.1 連結リストの概要

「連結(れんけつ)リスト」は、次の特徴を持つコンテナです。<br>
C++では、連結リストを`list`型として定義しています。

```cpp
template<typename T>
class list;
```

`list`型は、次のような特徴を持ちます。

* 追加、または削除する要素の位置が分かっている場合、どの位置であっても非常に高速に追加や削除ができる
* `[]`演算子が使えない
* イテレータの種類が「双方向イテレータ」のため、`++`または`--`演算子でしか位置を変えられない。

`list`型を使うには、`list`というヘッダファイルをインクルードします

次のプログラムは、`list`型の変数に対して要素を削除、追加する例です。

**コード**

```cpp
#include <iostream>
#include <list>
using namespace std;

int main() {
  list<int> v = { 1, 2, 3, 4, 5 };

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;

  // 先頭から2番目の要素を削除
  auto i = v.begin();
  i++;
  i++;
  v.erase(i);

  // 末尾の一つ前に10を追加
  i = v.end();
  i--;
  v.emplace(i, 10);

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;
}
```

**実行結果**

```txt
1 2 3 4 5
1 2 4 10 5
```


### 2.2 連結リストのデータ構造

`list`の各要素は、以下のような要素単位のメモが、メモ帳のあちこちに散らばっていると考えられます。

```txt
前の要素の位置: ◯ページ△行目
次の要素の位置: □ページ✕行目
データ: 123
```

データの型をテンプレート引数`T`(ティー)とすると、このメモは次のような構造体としてあらわせます。

```cpp
template<typename T>
struct ListData {
  ListData* prev; // 前の要素の位置
  ListData* next; // 次の要素の位置
  T data;         // データ
};
```

実際の`list`型の要素とは細部が異なりますが、おおよそはこの形をしていると考えてください。<br>
この種類のデータ構造が「連結リスト」と呼ばれるのは、`ListData`同士が、次の図のように互いに連結されているからです。

$$
list-\boxed{ListData}-\boxed{ListData}-\boxed{ListData}-...
$$

このように、連結リストの構造は、`vector`のような配列と比べて複雑です。<br>
そのため、格納されたデータを読み書きする、という基本的な用途だけを考えると、`vector`よりも性能が低くなります。



### 2.3 vectorとlistのどちらを使うべきか

基本的な考え方は「とりあえず`vector`を使おう」です。

#### listの利点

&emsp;先頭や中間部分の追加や削除は、`list`のほうが`vector`よりかなり高速です。<br>
&emsp;中間部分への追加や削除が多い場合は、`list`の使用を検討してください。

&emsp;さらに、`list`は「他の`list`から要素を移動」したり、「2つのリストをまとめる」ような操作も非常に高速です。<br>
&emsp;要素をつなぎ直すだけで済むからです。

#### listの欠点

&emsp;データを末尾に追加したり、末尾のデータを削除する処理は`vector`のほうが高速です。<br>
&emsp;このため、末尾への追加と削除しかしないなら、`list`より`vector`を使うべきです。

&emsp;別の欠点として、`ListData`は隣の要素の位置しか分からない、という事が挙げられます。<br>
&emsp;そのため、「N個先の要素」は、要素をひとつずつ辿っていく以外に知る方法はありません。<br>
&emsp;`list`型が`[]`演算子を使えず、イテレータの種類が双方向イテレータになっているのは、このデータ構造が原因です。

#### listよりvectorを使おう

&emsp;利点と欠点をまとめると、`list`が役に立つのは「リストの途中への追加や削除が頻繁に行われる場合」に限られます。<br>
&emsp;ですが、そういったケースは多くありません。そのため、基本的には`vector`を使うべき、となります。


### 2.4 データを検索する

`list`のデータを検索するには、汎用アルゴリズムの`find`関数を使います。

**コード**

```cpp
#include <iostream>
#include <list>
#include <algorithm>
using namespace std;

int main() {
  list<int> v = { 1, 2, 3, 4, 5 };

  auto i = find(v.begin(), v.end(), 3);
  if (i != v.end()) {
    cout << *i << "が見つかりました" << endl;
  } else {
    cout << "見つかりません" << endl;
  }
}
```

**実行結果**

```txt
3が見つかりました
```


### 2.5 データを追加する

`list`にデータを追加するには、位置に応じて以下の3つのメンバ関数を使い分けます。

>**書式**
>
>```cpp
>void push_back(const T& x);
>void push_front(const T& x);
>iterator emplace(iterator i, const T& x);
>```
>
>**引数**
>
>* x&emsp;追加するデータ
>* i&emsp;追加する位置を指すイテレータ
>
>**戻り値**
>
>`emplace`メンバ関数は、追加したデータを指すイテレータを返します。

データを末尾に追加する場合は`push_back`(プッシュ・バック)を使います。<br>
データを先頭に追加する場合は`push_front`(プッシュ・フロント)を使います。<br>
データを好きな位置に追加したい場合は`emplace`(エンプレイス)を使います。

次のプログラムは、３つの関数を使ってデータを追加する例です。

**コード**

```cpp
#include <iostream>
#include <list>
using namespace std;

int main() {
  list<int> v = { 1, 2, 3, 4, 5 };

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;

  // 末尾にデータを追加
  v.push_back(10);

  // 先頭にデータを追加
  v.push_front(20);

  // 先頭から2番目にデータを追加
  auto i = v.begin();
  i++;
  i++;
  v.emplace(i, 30);

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;
}
```

**実行結果**

```txt
1 2 3 4 5
20 1 30 2 3 4 5 10
```


### 2.6 データを削除する

`list`からデータを削除するには、用途に応じて次の3つのメンバ関数を使い分けます。

>**書式**
>
>```cpp
>iterator erase(iterator i);
>size_type remove(const T& x);
>
>template<typename P>
>size_type remove_if(P pred);
>```
>
>**引数**
>
>* i&emsp;削除する要素を指すイテレータ
>* x&emsp;削除するデータ
>* pred&emsp;削除する条件をあらわす関数
>
>**戻り値**
>
>`erase`メンバ関数は、削除したデータの次のデータ」を指すイテレータを返します。<br>
>`remove`メンバ関数および`remove_if`メンバ関数は、削除したデータの数を返します。

`erase`(イレース)メンバ関数は、「削除するデータの位置が分かっている」場合に使います。<br>
`remove`(リムーブ)メンバ関数は、「削除するデータが分かっている」場合に使います。<br>
`remove_if`(リムーブ・イフ)メンバ関数は、「削除する条件が分かっている」場合に使います。

次のプログラムは、3つのメンバ関数を使ってデータを削除する例です。

**コード**

```cpp
#include <iostream>
#include <list>
using namespace std;

void print(const list<int>& v) {
  for (auto i = v.begin(); i != v.end(); i++) {
    cout << *i << " ";
  }
  cout << endl;
}

int main() {
  list<int> v = { 1, 2, 3, 4, 5, 4, 3, 2 };
  cout << "削除前" << endl;
  print(v);

  auto i = find(v.begin(), v.end(), 2);
  auto j = v.erase(i);
  cout << "削除したデータの次のデータは:" << *j << endl;
  print(v);

  size_t a = v.remove(4);
  cout << "4を" << a << "個削除" << endl;
  print(v);

  size_t b = v.remove_if([](int n){ return n % 2; });
  cout << "奇数を" << b << "個削除" << endl;
  print(v);
}
```

**実行結果**

```txt
削除前
1 2 3 4 5 4 3 2
削除したデータの次のデータは:3
1 3 4 5 4 3 2
4を2個削除
1 3 5 3 2
奇数を4個削除
2
```


### 2.7 データを操作するメンバ関数について

`list`型は複雑な構造なので、コンテナを変更するような操作はメンバ関数として用意されています。

| メンバ関数名 | 機能 |
|:-------------|:-----|
| <font size=3>splice</font>(スプライス) | 他のコンテナから要素を移動する |
| <font size=3>unique</font>(ユニーク) | 重複した要素をコンテナから削除する |
| <font size=3>merge</font>(マージ) | 2つのコンテナを併合する |
| <font size=3>sort</font>(ソート) | コンテナを並べ替える |
| <font size=3>reverse</font>(リバース) | コンテナを反転する |

すべてを説明している余裕はないので、ここでは「こういう機能がある」という紹介にとどめます。<br>
詳しくは`cpprefjp.github.io`などのリファレンスを参照してください。

実際のところ、これらの操作が必要となることは多くありません。<br>
ですが、「`list`型はメンバ関数を使う操作が多い」ということは覚えておくとよいでしょう。


----

## 4 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


### ❓️問題１

1個の文字列が入力されます。<br>
入力された文字列が`dog`のときは`ワンワン`、`cat`のときは`ニャーニャー`、それ以外のときは`ガオー`と出力するプログラムを作成しなさい。

| 入力 | 出力する文字列 |
|:----:|:---------------|
| <font size=2>dog</font> | ワンワン     |
| <font size=2>cat</font> | ニャーニャー |
| その他                  | ガオー       |

<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1. `string`型の変数`s`を定義する
2. if文を使って、`s`が文字列`dog`と等しい場合は、標準出力`cout`に文字列`ワンワン`を出力する
3. else if文を使って、`s`が文字列`cat`と等しい場合は、標準出力`cout`に文字列`ニャーニャー`を出力する
4. else句を使って、標準出力`cout`に文字列`ガオー`を出力する

</details><br>

**入力データ例（１）**

```
dog
```

**出力例（１）**

```txt
ワンワン
```

**入力データ例（２）**

```
cat
```

**出力例（２）**

```txt
ニャーニャー
```

**入力データ例（３）**

```
doggie
```

**出力例（３）**

```txt
ガオー
```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
#include <string>
using namespace std;

int main() {
  // この下に、文字列を読み込んで鳴き声を出力するプログラムを書く

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "この下をクリックして、動物の名前を入力" && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "ワンワン\nニャーニャー\nガオー") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && echo "dog" | ./practice_01a && echo "cat" | ./practice_01a && echo "human" | ./practice_01a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
#include <string>
using namespace std;

int main() {
  // この下に、文字列を読み込んで鳴き声を出力するプログラムを書く
  string s;
  cin >> s;

  if (s == "dog") {
    cout << "ワンワン" << endl;
  } else if (s == "cat") {
    cout << "ニャーニャー" << endl;
  } else {
    cout << "ガオー" << endl;
  }
}